# Cargar clasificador ya hecho

In [40]:
import numpy as np
import transformers

import shap

In [41]:
classifier = transformers.pipeline("sentiment-analysis", return_all_scores=True)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
`return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.


## Prueba

In [42]:
classifier("This restaurant is awesome")

[[{'label': 'NEGATIVE', 'score': 0.00012563355267047882},
  {'label': 'POSITIVE', 'score': 0.9998743534088135}]]

# Carga Tweets

## Cargar tweets con palabras claves

In [21]:
import csv

# Ruta del archivo CSV
csv_file_path = 'dataset/Tweets.csv'

# Palabras clave para buscar tweets
keywords = ['happy', 'sad', 'ever', 'love', 'can']

# Número máximo de tweets que deseas recuperar por palabra clave
max_tweets_per_keyword = 10

# Listas para almacenar los tweets por cada palabra clave
tweets_por_palabra_clave = {keyword: [] for keyword in keywords}

# Leer tweets desde el archivo CSV
with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        text = row['text']
        sentiment = row['sentiment']
        for keyword in keywords:
            if keyword in text.lower():
                tweets_por_palabra_clave[keyword].append(text)
                max_tweets_per_keyword -= 1
                if max_tweets_per_keyword == 0:
                    break
        if max_tweets_per_keyword == 0:
            break

# Imprimir los tweets almacenados en las listas
for keyword, tweets in tweets_por_palabra_clave.items():
    print(f"Tweets para '{keyword}':")
    for tweet in tweets:
        print(tweet)
    print()


Tweets para 'happy':
 If it is any consolation I got my BMI tested hahaha it says I am obesed  well so much for being unhappy for about 10 minutes.
A little happy for the wine jeje ok it`sm my free time so who cares, jaja i love this day
 Car not happy, big big dent in boot! Hoping theyre not going to write it off, crossing fingers and waiting

Tweets para 'sad':
 Sooo SAD I will miss you here in San Diego!!!

Tweets para 'ever':
 as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff
is back home now      gonna miss every one

Tweets para 'love':
 as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff
I really really like the song Love Story by Taylor Swift
A little happy for the wine jeje ok it`sm my free time so who cares, jaja i love this day

Tweets para 'can':
Playing Ghost Online is really interesting. The new updates are Kirin pet and Metamorph for third job.  Can`t wait to h

## Cargar tweets dudosos

In [24]:
i=0
# Leer tweets desde el archivo CSV y realizar análisis de sentimiento
with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        if i>3:
            break
        text = row['text']
        sentiment = row['sentiment']
        # Realizar análisis de sentimiento
        result = classifier(text)
        # Obtener la probabilidad de ser positivo y negativo
        positive_prob = result[0][0]['score'] if result[0][0]['label'] == 'POSITIVE' else result[0][1]['score']
        negative_prob = result[0][0]['score'] if result[0][0]['label'] == 'NEGATIVE' else result[0][1]['score']

        # Verificar si las probabilidades de ser positivo y negativo son similares
        if abs(positive_prob - negative_prob) < 0.1:
            i+=1
            print("Tweet con probabilidades similares:")
            print(f"Texto: {text}")
            print(f"Sentimiento: {sentiment}")
            print(f"Probabilidad de ser positivo: {positive_prob}")
            print(f"Probabilidad de ser negativo: {negative_prob}")
            print()

Tweet con probabilidades similares:
Texto: MAYDAY?!
Sentimiento: neutral
Probabilidad de ser positivo: 0.455401211977005
Probabilidad de ser negativo: 0.5445987582206726

Tweet con probabilidades similares:
Texto: Screw the reviews, I thought Wolverine was awesome. But not enough Dominic Monaghan for my liking.
Sentimiento: positive
Probabilidad de ser positivo: 0.5355139970779419
Probabilidad de ser negativo: 0.4644860029220581

Tweet con probabilidades similares:
Texto:  Thx
Sentimiento: neutral
Probabilidad de ser positivo: 0.5395320653915405
Probabilidad de ser negativo: 0.4604679346084595

Tweet con probabilidades similares:
Texto: _Live That`s what I want. More the better. Bound to be a few bad eggs though, but they will soon learn.
Sentimiento: neutral
Probabilidad de ser positivo: 0.5029829740524292
Probabilidad de ser negativo: 0.4970170259475708



## Visualizaciones ejemplo

In [14]:
classifier("If it is any consolation I got my BMI tested hahaha it says I am obesed  well so much for being unhappy for about 10 minutes")

[[{'label': 'NEGATIVE', 'score': 0.9976692795753479},
  {'label': 'POSITIVE', 'score': 0.0023306654766201973}]]

In [15]:
classifier("A little happy for the wine jeje ok it`sm my free time so who cares, jaja i love this day")

[[{'label': 'NEGATIVE', 'score': 0.00022044003708288074},
  {'label': 'POSITIVE', 'score': 0.9997795224189758}]]

In [17]:
classifier("Sooo SAD I will miss you here in San Diego!!!")

[[{'label': 'NEGATIVE', 'score': 0.014478222467005253},
  {'label': 'POSITIVE', 'score': 0.9855217933654785}]]

In [18]:
classifier("as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff")

[[{'label': 'NEGATIVE', 'score': 0.9979243278503418},
  {'label': 'POSITIVE', 'score': 0.002075662836432457}]]

In [19]:
classifier("I really really like the song Love Story by Taylor Swift")

[[{'label': 'NEGATIVE', 'score': 0.00032968970481306314},
  {'label': 'POSITIVE', 'score': 0.9996702671051025}]]

In [22]:
classifier("Can`t access your site!")

[[{'label': 'NEGATIVE', 'score': 0.999301552772522},
  {'label': 'POSITIVE', 'score': 0.0006984112551435828}]]

In [25]:
classifier("Screw the reviews, I thought Wolverine was awesome. But not enough Dominic Monaghan for my liking.")

[[{'label': 'NEGATIVE', 'score': 0.4644860029220581},
  {'label': 'POSITIVE', 'score': 0.5355139970779419}]]

# Interpretabilidad

## Ejemplo Positivo

A little happy for the wine jeje ok it`sm my free time so who cares, jaja i love this day

In [27]:
sentence="A little happy for the wine jeje ok it`sm my free time so who cares, jaja i love this day"

### LIME

In [32]:
import lime
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=['NEGATIVE', 'POSITIVE'])

In [46]:
exp = explainer.explain_instance(sentence, classifier.predict, num_samples=10, labels=[0,1])

TypeError: list indices must be integers or slices, not tuple

In [35]:
exp = explainer.explain_instance(sentence, classifier.predict)

TypeError: list indices must be integers or slices, not tuple